In [1]:
import numpy as np
import pandas as pd
from torch.distributions.gumbel import *
from scipy.sparse import csr_matrix
import scipy.sparse as sp
import datetime
import torch
from torch import nn, optim
import sys
import time
import random
import os
from tqdm import tqdm

In [2]:
def seed_torch(seed=6298):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)
    #torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    #torch.backends.cudnn.benchmark = False
    #torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.enabled = False
seed_torch()

In [3]:
!export CUDA_VISIBLE_DEVICES=0,1
cuda = torch.device('cuda') 
#torch.cuda.set_device(1)

# DataLoader

In [12]:
def convert_sp_mat_to_sp_tensor(X):
    coo = X.tocoo().astype(np.float32)
    row = torch.Tensor(coo.row).long()
    col = torch.Tensor(coo.col).long()
    index = torch.stack([row, col])
    data = torch.FloatTensor(coo.data)
    return torch.sparse.FloatTensor(index, data, torch.Size(coo.shape))

def getSparseGraph(n_users, m_items, Network):
    #print("generating adjacency matrix")
    adj_mat = sp.dok_matrix((n_users + m_items, n_users + m_items), dtype=np.float32)
    adj_mat = adj_mat.tolil()
    R = Network.tolil()
    adj_mat[:n_users, n_users:] = R
    adj_mat[n_users:, :n_users] = R.T
    adj_mat = adj_mat.todok()
    
    rowsum = np.array(adj_mat.sum(axis=1))
    d_inv = np.power(rowsum, -0.5).flatten()
    d_inv[np.isinf(d_inv)] = 0.
    d_mat = sp.diags(d_inv)
    
    norm_adj = d_mat.dot(adj_mat)
    norm_adj = norm_adj.dot(d_mat)
    norm_adj = norm_adj.tocsr()
    
    Graph = convert_sp_mat_to_sp_tensor(norm_adj)
    Graph = Graph.coalesce()
    return Graph

# Evaluation
def RecallPrecision_ATk(test_data, r, k):
    """
    test_data should be a list? cause users may have different amount of pos items. shape (test_batch, k)
    pred_data : shape (test_batch, k) NOTE: pred_data should be pre-sorted
    k : top-k
    """
    right_pred = r[:, :k].sum(1)
    precis_n = k
    recall_n = np.array([len(test_data[i]) for i in range(len(test_data))])
    recall = np.sum(right_pred/recall_n)
    precis = np.sum(right_pred)/precis_n
    return {'recall': recall, 'precision': precis}

def NDCGatK_r(test_data,r,k):
    """
    Normalized Discounted Cumulative Gain
    rel_i = 1 or 0, so 2^{rel_i} - 1 = 1 or 0
    """
    assert len(r) == len(test_data)
    pred_data = r[:, :k]

    test_matrix = np.zeros((len(pred_data), k))
    for i, items in enumerate(test_data):
        length = k if k <= len(items) else len(items)
        test_matrix[i, :length] = 1
    max_r = test_matrix
    idcg = np.sum(max_r * 1./np.log2(np.arange(2, k + 2)), axis=1)
    dcg = pred_data*(1./np.log2(np.arange(2, k + 2)))
    dcg = np.sum(dcg, axis=1)
    idcg[idcg == 0.] = 1.
    ndcg = dcg/idcg
    ndcg[np.isnan(ndcg)] = 0.
    return np.sum(ndcg)

def getLabel(test_data, pred_data):
    r = []
    for i in range(len(test_data)):
        groundTrue = test_data[i]
        predictTopK = pred_data[i]
        pred = list(map(lambda x: x in groundTrue, predictTopK))
        pred = np.array(pred).astype("float")
        r.append(pred)
    return np.array(r).astype('float')

def test_one_batch(X):
    sorted_items = X[0].numpy()
    groundTrue = X[1]
    r = getLabel(groundTrue, sorted_items)
    pre, recall, ndcg = [], [], []
    for k in config["topks"]:
        ret = RecallPrecision_ATk(groundTrue, r, k)
        pre.append(ret['precision'])
        recall.append(ret['recall'])
        ndcg.append(NDCGatK_r(groundTrue,r,k))
    return {'recall':np.array(recall), 
            'precision':np.array(pre), 
            'ndcg':np.array(ndcg)}

def Test(if_eval):
    Recmodel.eval()
    u_batch_size = config['test_u_batch_size']
    max_K = max(config["topks"])
    multicore = config['multicore']
    if multicore == 1:
        pool = multiprocessing.Pool(12)
    results = {'precision': np.zeros(len(config["topks"])),
               'recall': np.zeros(len(config["topks"])),
               'ndcg': np.zeros(len(config["topks"]))}
    with torch.no_grad():
        users = list(dict_interactions.keys())
        users_list = []
        rating_list = []
        groundTrue_list = []
        total_batch = len(users) // u_batch_size + 1
        for batch_users in minibatch(np.arange(n_users_t), batch_size=u_batch_size):
            allPos = getUserPosItems(batch_users)
            if if_eval:
                groundTrue = [dict_interactions[u][-2:-1] for u in batch_users]
            else:
                groundTrue = [dict_interactions[u][-1:] for u in batch_users]
            batch_users = torch.Tensor(batch_users).long()
            
            rating = Recmodel.getUsersRating_t(batch_users.cuda())
            #rating = Recmodel.getUsersRating_t(batch_users)
            
            exclude_index = []
            exclude_items = []
            for range_i, items in enumerate(allPos):
                exclude_index.extend([range_i] * len(items))
                exclude_items.extend(items)
            rating[exclude_index, exclude_items] = -(1<<10)
            _, rating_k = torch.topk(rating, k=max_K)
            
            del rating
            users_list.append(batch_users)
            rating_list.append(rating_k)
            groundTrue_list.append(groundTrue)
        X = zip(rating_list, groundTrue_list)
        if multicore == 1:
            pre_results = pool.map(test_one_batch, X)
        else:
            pre_results = []
            for x in X:
                pre_results.append(test_one_batch(x))
        for result in pre_results:
            results['recall'] += result['recall']
            results['precision'] += result['precision']
            results['ndcg'] += result['ndcg']
        results['recall'] /= float(len(users))
        results['precision'] /= float(len(users))
        results['ndcg'] /= float(len(users))
        
        if multicore == 1:
            pool.close()
        #print(results)
        return results

def minibatch(*tensors, batch_size):

    if len(tensors) == 1:
        tensor = tensors[0]
        for i in range(0, len(tensor), batch_size):
            yield tensor[i:i + batch_size]
    else:
        for i in range(0, len(tensors[0]), batch_size):
            yield tuple(x[i:i + batch_size] for x in tensors)

def getUserPosItems(batch_users):
    posItems = []
    for user in batch_users:
        posItems.append(dict_interactions[user][:-2])
    return posItems

def shuffle(*arrays, **kwargs):

    require_indices = kwargs.get('indices', False)

    if len(set(len(x) for x in arrays)) != 1:
        raise ValueError('All inputs to shuffle must have '
                         'the same length.')

    shuffle_indices = np.arange(len(arrays[0]))
    np.random.shuffle(shuffle_indices)

    if len(arrays) == 1:
        result = arrays[0][shuffle_indices]
    else:
        result = tuple(x[shuffle_indices] for x in arrays)

    if require_indices:
        return result, shuffle_indices
    else:
        return result

def UniformSample_original():
    """
    the original impliment of BPR Sampling in LightGCN
    :return:
        np.array
    """
    user_num_t = len(tr_u_t) - len(tr_u_t)%config["bpr_batch_size"]
    users_t = np.random.randint(0, n_users_t, user_num_t)
    users_s = np.random.randint(0, n_users_s, user_num_t)
    S = []
    for i, user in enumerate(users_t):
        posForUser = dict_interactions[user][:-2]
        if len(posForUser) == 0:
            continue
        posindex = np.random.randint(0, len(posForUser))
        positem = posForUser[posindex]
        while True:
            negitem = np.random.randint(0, m_items_t)
            if negitem in posForUser:
                continue
            else:
                break           
        
        posForUser_s = dict_interactions_s[users_s[i]]
        if len(posForUser_s) == 0:
            continue
        posindex = np.random.randint(0, len(posForUser_s))
        positem_s = posForUser_s[posindex]
        while True:
            negitem_s = np.random.randint(0, m_items_s)
            if negitem in posForUser:
                continue
            else:
                break                 
        S.append([user, positem, negitem, users_s[i], positem_s, negitem_s])
    return np.array(S)

# Utils

In [12]:
class BPRLoss:
    def __init__(self):
        self.decay_reg = config['decay_reg']     # beta
        self.decay_dp = config['decay_dp'] # gamma
        self.decay_kt = config['decay_kt'] # alpha
        
        self.lr = config['lr']
        self.opt = optim.Adam(Recmodel.parameters(), lr=self.lr)

    def stageOne(self, users, posI, negI, users_s, posI_s, negI_s):
        #pred_loss_s, pred_loss_t, dp_loss_s, dp_loss_t , kld_loss_t, reg_loss = Recmodel.bpr_loss(users, posI, negI, 
        #                                                                                users_s, posI_s, negI_s)
        pred_loss_s, pred_loss_t, dp_loss_s, dp_loss_t , kld_loss_t, reg_loss = Recmodel.bpr_loss(users.cuda(), posI.cuda(), negI.cuda(), 
                                                                                        users_s.cuda(), posI_s.cuda(), negI_s.cuda())
        dp_loss_s = dp_loss_s * self.decay_dp
        dp_loss_t = dp_loss_t * self.decay_dp
        
        kld_loss_t =kld_loss_t * self.decay_kt
        reg_loss = reg_loss * self.decay_reg
        
        #loss = pred_loss_s + pred_loss_t + dp_loss_s + dp_loss_t + kld_loss_t + reg_loss
        loss = pred_loss_s + pred_loss_t + kld_loss_t + reg_loss
        
        self.opt.zero_grad()
        loss.backward()
        self.opt.step()

        return loss.cpu().item(), pred_loss_s.cpu().item(), pred_loss_t.cpu().item(), \
                dp_loss_s.cpu().item(), dp_loss_t.cpu().item(), kld_loss_t.cpu().item(), reg_loss.cpu().item()
    
def BPR_train_original(epoch):
    Recmodel.train()
    bpr = BPRLoss()
    
    T = UniformSample_original()
    
    users = torch.Tensor(T[:, 0]).long()
    posItems = torch.Tensor(T[:, 1]).long()
    negItems = torch.Tensor(T[:, 2]).long()
    
    users_s = torch.Tensor(T[:, 3]).long()
    posItems_s = torch.Tensor(T[:, 4]).long()
    negItems_s = torch.Tensor(T[:, 5]).long()
    
    users, posItems, negItems, users_s, posItems_s, negItems_s = shuffle(users, posItems, negItems, users_s, posItems_s, negItems_s)
    
    total_batch = len(users) // config['bpr_batch_size'] + 1
    
    aver_loss, aver_pre_loss_s, aver_pre_loss_t, aver_rec_loss_s, aver_rec_loss_t, aver_kt_loss, aver_reg_loss = 0., 0., 0., 0., 0., 0.,0.
    for (batch_i, 
         (batch_users, 
          batch_posItem, 
          batch_negItem, 
          batch_users_s, 
          batch_posItem_s,
          batch_negItem_s)) in enumerate(minibatch(users, 
                                                    posItems,
                                                    negItems, 
                                                    users_s, 
                                                    posItems_s, 
                                                    negItems_s,
                                                    batch_size=config['bpr_batch_size'])):
        
        loss, pred_loss_s, pred_loss_t, dp_loss_s, dp_loss_t , kld_loss_t, reg_loss = bpr.stageOne(batch_users,batch_posItem, batch_negItem, 
                                                                                         batch_users_s, batch_posItem_s,batch_negItem_s)
        aver_loss += loss
        aver_pre_loss_s += pred_loss_s
        aver_pre_loss_t += pred_loss_t
        aver_rec_loss_s += dp_loss_s
        aver_rec_loss_t += dp_loss_t
        aver_kt_loss += kld_loss_t
        aver_reg_loss += reg_loss

    aver_loss = aver_loss / total_batch
    aver_pre_loss_s = aver_pre_loss_s / total_batch
    aver_pre_loss_t = aver_pre_loss_t / total_batch
    aver_rec_loss_s = aver_rec_loss_s / total_batch
    aver_rec_loss_t = aver_rec_loss_t / total_batch
    aver_kt_loss = aver_kt_loss / total_batch
    aver_reg_loss = aver_reg_loss / total_batch
    
    return aver_loss, aver_pre_loss_s, aver_pre_loss_t, aver_rec_loss_s, aver_rec_loss_t, aver_kt_loss, aver_reg_loss

# SRTrans

class SRTrans(nn.Module):
    def __init__(self, 
                 config:dict,
                 UIGraph_s,
                 UIGraph_t,
                is_GNN):
        super(SRTrans, self).__init__()
        self.config = config
        self.UIGraph_s = UIGraph_s.cuda()
        self.UIGraph_t = UIGraph_t.cuda()
        self.__init_weight()
        self.is_GNN = is_GNN
        

    def __init_weight(self):
        self.n_users_s, self.n_users_t = n_users_s, n_users_t
        self.m_items_s, self.m_items_t = m_items_s, m_items_t

        self.latent_dim = self.config['latent_dim_rec']
        self.n_layers_s = self.config['lightGCN_n_layers_s']
        self.n_layers_t = self.config['lightGCN_n_layers_t']
        
        self.embedding_user_s = torch.nn.Embedding(
            num_embeddings=self.n_users_s, embedding_dim=self.latent_dim).cuda()
        
        self.embedding_item_s = torch.nn.Embedding.from_pretrained(
            torch.FloatTensor(list(dict_ItemIndex2vec_s.values())), freeze=True).cuda()
        
        self.embedding_user_t = torch.nn.Embedding(
            num_embeddings=self.n_users_t, embedding_dim=self.latent_dim).cuda()
        
        self.embedding_item_t = torch.nn.Embedding.from_pretrained(
            torch.FloatTensor(list(dict_ItemIndex2vec_t.values())), freeze=True).cuda()
        
        nn.init.normal_(self.embedding_user_s.weight, std=0.1)
        nn.init.normal_(self.embedding_user_t.weight, std=0.1)
        #-------------------------------------------------------------
        
        
        #self.fc_i_s = nn.Linear(768, self.latent_dim)
        self.fc_i_t = nn.Linear(768, self.latent_dim).cuda()
        #nn.init.normal_(self.fc_i_s.weight, std=0.1)
        #nn.init.normal_(self.fc_i_s.bias, std=0.1)  
        #nn.init.normal_(self.fc_i_t.weight, std=0.1)
        #nn.init.normal_(self.fc_i_t.bias, std=0.1)        
        #-------------------------------------------------------------      
        
        self.f = nn.Sigmoid()
        #self.UIGraph_s = getSparseGraph(self.n_users_s, self.m_items_s, UseritemNet_s)
        #self.UIGraph_t = getSparseGraph(self.n_users_t, self.m_items_t, UseritemNet_t)
        #--------------------------------------------------------------
                
        #self.l_clust_encoder_s = nn.Linear(self.config['latent_dim_rec'], self.config["n_cluster"])
        self.l_clust_encoder_t = nn.Linear(self.config['latent_dim_rec'], self.config['latent_dim_rec']).cuda()
        self.l_clust_encoder_t2 = nn.Linear(self.config['latent_dim_rec'], self.config["n_cluster"]).cuda()
        #nn.init.normal_(self.l_clust_encoder.weight, std=0.1)
        #nn.init.normal_(self.l_clust_encoder.bias, std=0.1)
        self.g_k = Gumbel(torch.zeros(config['bpr_batch_size'], self.config["n_cluster"]).cuda(), 
                          torch.ones(config['bpr_batch_size'], self.config["n_cluster"]).cuda())
        self.soft = nn.Softmax(dim=1)
        
        #--------------------------------------------------------------
        self.W_pp = nn.Linear(self.config['latent_dim_rec'], self.config['latent_dim_rec'], bias=False).cuda()
        #self.W_pe_s = nn.Linear(self.config['latent_dim_rec'], self.config['latent_dim_rec'])
        self.W_pe_t = nn.Linear(self.config['latent_dim_rec'], self.config['latent_dim_rec'], bias=False).cuda()
        
        nn.init.normal_(self.W_pp.weight, std=0.1)
        #nn.init.normal_(self.W.bias, std=0.1)       
        nn.init.normal_(self.W_pe_t.weight, std=0.1)
        #nn.init.normal_(self.W.bias, std=0.1)
        self.relu = nn.ReLU()
        
        #Dense_layers = [self.fc_i_s, self.fc_i_t, self.l_clust_encoder_s, self.l_clust_encoder_t]
        Dense_layers = [self.fc_i_t, self.l_clust_encoder_t, self.l_clust_encoder_t2]
        self.initial_denseLayers(Dense_layers)
        
        
        
    def initial_denseLayers(self, layer_lst):
        for layer in layer_lst:
            nn.init.normal_(layer.weight, std=0.1)
            nn.init.normal_(layer.bias, std=0.1)
        
    def computer_s(self):
        """
        propagate methods for lightGCN
        """       
        users_emb = self.embedding_user_s.weight
        items_emb = self.fc_i_t(self.embedding_item_s.weight)
        

        all_emb = torch.cat([users_emb, items_emb])

        embs = [all_emb]
    
        g_droped = self.UIGraph_s
        
        for layer in range(self.n_layers_s):
            all_emb = torch.sparse.mm(g_droped, all_emb)
            embs.append(all_emb)
        embs = torch.stack(embs, dim=1)
        light_out = torch.mean(embs, dim=1)
        users, items = torch.split(light_out, [self.n_users_s, self.m_items_s])
        '''
        users, items = users_emb, items_emb
        '''        
        return users, items
    
    def computer_t(self):
        """
        propagate methods for lightGCN
        """       
        users_emb = self.embedding_user_t.weight
        items_emb = self.fc_i_t(self.embedding_item_t.weight)
        if self.is_GNN:
            all_emb = torch.cat([users_emb, items_emb])

            embs = [all_emb]

            g_droped = self.UIGraph_t

            for layer in range(self.n_layers_t):
                all_emb = torch.sparse.mm(g_droped, all_emb)
                embs.append(all_emb)
            embs = torch.stack(embs, dim=1)
            light_out = torch.mean(embs, dim=1)
            users, items = torch.split(light_out, [self.n_users_t, self.m_items_t])
        else:
            users, items = users_emb, items_emb
        
        return users, items
    
    def cluster(self, x, tao, l_clust_encoder, l_clust_encoder_2):
        
        alpha = self.relu(l_clust_encoder(x))
        alpha = l_clust_encoder_2(alpha)
        y = self.soft((alpha+self.g_k.sample())/tao) 
        #y = y / (y.sum(dim=0)+0.0000001)
        
        return torch.matmul(y.t(),x)
       
    def knowledge_transfer(self, p_s, p_t, gamma_kt, W):
        # message pass from paper [https://arxiv.org/pdf/1704.01212.pdf]
        
        h = torch.cat([p_s, p_t]) / gamma_kt
        I = torch.eye(h.shape[0]).cuda()
        
        #Sim = torch.cdist(h,h,p=2)/2
        a_norm = h / (h.norm(dim=1)[:, None]+0.0000001)
        Sim = torch.mm(a_norm, a_norm.transpose(0,1))
        
        A = self.soft(Sim) + I
        D = torch.diag(1/torch.pow(A.sum(dim=1), 1/2))

        L = torch.matmul(torch.matmul(D, A), D)
        #L = D - A
        h_next = torch.matmul(L,h)
        h_next = self.relu(W(h_next))
        
        p_s_bar, p_t_bar = h_next[:p_s.shape[0]], h_next[p_s.shape[0]:]
        return p_s_bar, p_t_bar, A
    
    def knowledge_transfer_pp(self, p_s, p_t, p0_s, p0_t, gamma_kt, W):
        # message pass from paper [https://arxiv.org/pdf/1704.01212.pdf]
        
        h = torch.cat([p_s, p_t]) / gamma_kt
        I = torch.eye(h.shape[0]).cuda()
        
        #Sim = torch.cdist(h,h,p=2)/2
        a_norm = h / (h.norm(dim=1)[:, None]+0.0000001)
        Sim = torch.mm(a_norm, a_norm.transpose(0,1))     
        
        A = self.soft(Sim) + I
        D = torch.diag(1/torch.pow(A.sum(dim=1), 1/2))

        L = torch.matmul(torch.matmul(D, A), D)
        #L = D - A
        h0 = torch.cat([p0_s, p0_t]) / gamma_kt
        h_next = torch.matmul(L,h0)
        h_next = self.relu(W(h_next))
        
        p_s_bar, p_t_bar = h_next[:p_s.shape[0]], h_next[p_s.shape[0]:]
        return p_s_bar, p_t_bar, A
        
    def getUsersRating_t(self, users):
        all_users, all_items = self.computer_t()
        
        
        users_emb = all_users[users.long()] 
        items_emb = all_items
        
        rating = self.f(torch.matmul(users_emb, items_emb.t()))
        return rating.cpu()
    
    def getEmbedding_s(self, users, pos_items, neg_items):
        all_users, all_items = self.computer_s()
        
        users_emb= all_users[users.long()]
        
        pos_emb = all_items[pos_items]
        neg_emb = all_items[neg_items]
        
        users_emb_ego = self.embedding_user_s(users)  
        pos_emb_ego = self.fc_i_t(self.embedding_item_s(pos_items))
        
        return users_emb, pos_emb, neg_emb, users_emb_ego, pos_emb_ego
    
    def getEmbedding_t(self, users, pos_items, neg_items):
        all_users, all_items = self.computer_t()
        
        users_emb= all_users[users.long()]
        
        pos_emb = all_items[pos_items]
        neg_emb = all_items[neg_items]
        
        users_emb_ego = self.embedding_user_t(users)  
        pos_emb_ego = self.fc_i_t(self.embedding_item_t(pos_items))
        
        return users_emb, pos_emb, neg_emb, users_emb_ego, pos_emb_ego
    
    def pred_loss(self, users_emb, pos_emb, neg_emb):
        
        poscores = torch.mul(users_emb, pos_emb).sum(dim=1)
        negcores = torch.mul(users_emb, neg_emb).sum(dim=1)
        
        loss = torch.mean(nn.functional.softplus(negcores - poscores))
        
        return loss
    
    def dp_loss(self, posI, negI, embedding_item, fc_i):
        pos_I_ori = embedding_item(posI.long()) 
        neg_I_ori = embedding_item(negI.long())    
        
        I_sim_ori = torch.cosine_similarity(pos_I_ori, neg_I_ori, dim=1)
        I_sim = torch.cosine_similarity(fc_i(pos_I_ori), fc_i(neg_I_ori), dim=1)  
        
        loss = (I_sim_ori - I_sim).pow(2).mean()
        
        return loss
    
    def kld_loss(self, users_emb, pos_emb, pos_emb_bar):
        
        y = self.f(torch.mul(users_emb, pos_emb).sum(dim=1))
        y_bar = self.f(torch.mul(users_emb, pos_emb_bar).sum(dim=1))

        
        KLD_loss = y_bar * torch.log((y_bar+0.00001)/(y+0.00001)) + (1.0001 - y_bar) * torch.log((1.00001 - y_bar)/(1.00001 - y))
        KLD_loss = KLD_loss.mean()

        
        return KLD_loss
    
    def bpr_loss(self, users_t, posI_t, negI_t, users_s, posI_s, negI_s):
        users_emb_s, pos_emb_s, neg_emb_s, userEmb0_s, pos_Emb0_s = self.getEmbedding_s(users_s.long(), posI_s.long(), negI_s.long())
        users_emb_t, pos_emb_t, neg_emb_t, userEmb0_t, pos_Emb0_t = self.getEmbedding_t(users_t.long(), posI_t.long(), negI_t.long())
        pred_loss_s = self.pred_loss(users_emb_s, pos_emb_s, neg_emb_s)
        pred_loss_t = self.pred_loss(users_emb_t, pos_emb_t, neg_emb_t)
        
        dp_loss_s = self.dp_loss(posI_s, negI_s, self.embedding_item_s, self.fc_i_t)
        dp_loss_t = self.dp_loss(posI_t, negI_t, self.embedding_item_t, self.fc_i_t)
          
        #---------------------------------------------------------------------------------
        #KL_divergence between predictions and knowledge enhanced predictions
        
        p_s = self.cluster(pos_Emb0_s, self.config["tao"], self.l_clust_encoder_t, self.l_clust_encoder_t2)
        p_t = self.cluster(pos_Emb0_t, self.config["tao"], self.l_clust_encoder_t, self.l_clust_encoder_t2) 
        
        p0_s = self.cluster(pos_emb_s, self.config["tao"], self.l_clust_encoder_t, self.l_clust_encoder_t2)
        p0_t = self.cluster(pos_emb_t, self.config["tao"], self.l_clust_encoder_t, self.l_clust_encoder_t2)  
        
        p_s_bar, p_t_bar, _ = self.knowledge_transfer_pp(p_s, p_t, p0_s, p0_t, self.config["gamma_kt"], self.W_pp)   
        
        pos_emb_bar_t, _, _ = self.knowledge_transfer(pos_emb_t, p_t_bar, self.config["gamma_kt"], self.W_pe_t)
        
        kld_pos_loss_t = self.kld_loss(users_emb_t, pos_emb_t, pos_emb_bar_t)
        kld_loss_t = kld_pos_loss_t 
        
        
        #neg_emb_bar_t, _, _ = self.knowledge_transfer(neg_emb_t, p_t_bar, self.config["gamma_kt"], self.W_pe_t)
        #kld_neg_loss_t = self.kld_loss(users_emb_t, neg_emb_t, neg_emb_bar_t)
        #kld_loss_t = kld_pos_loss_t + kld_neg_loss_t
        
        #kld_loss_t = torch.Tensor([0.])
        
        reg_loss = (1/2)*(userEmb0_t.norm(2).pow(2)/float(self.config['bpr_batch_size']*self.latent_dim) + \
                          userEmb0_s.norm(2).pow(2)/float(self.config['bpr_batch_size']*self.latent_dim) + \
                          #p_s.norm(2).pow(2)/float(self.config['n_cluster']*self.latent_dim) + \
                          #p_t.norm(2).pow(2)/float(self.config['n_cluster']*self.latent_dim) + \
                          self.fc_i_t.weight.norm(2).pow(2)/float(768*self.latent_dim) + \
                          self.fc_i_t.bias.norm(2).pow(2)/float(self.latent_dim) + \
                          self.l_clust_encoder_t.weight.norm(2).pow(2)/float(self.latent_dim * self.config["n_cluster"]) + \
                          self.l_clust_encoder_t2.weight.norm(2).pow(2)/float(self.latent_dim * self.config["n_cluster"]) + \
                          self.W_pp.weight.norm(2).pow(2)/float(self.latent_dim * self.latent_dim) + \
                          self.W_pe_t.weight.norm(2).pow(2)/float(self.latent_dim * self.latent_dim)) 
                            
        
        return pred_loss_s, pred_loss_t, dp_loss_s, dp_loss_t , kld_loss_t, reg_loss
       
    def forward(self, users, items, clusters):
        # compute embedding
        all_users, all_items = self.computer()
        # print('forward')
        #all_users, all_items = self.computer()
        users_emb = all_users[users]
        items_emb = all_items[items]
        inner_pro = torch.mul(users_emb, items_emb)
        gamma     = torch.sum(inner_pro, dim=1)
        
        # compute embedding
        all_users_reg, all_clusters_reg = self.computerClusterReg()
        # print('forward')
        #all_users, all_items = self.computer()
        users_emb_reg = all_users_reg[users]
        clusters_emb_reg = all_clusters_reg[clusters]
        inner_pro_reg = torch.mul(users_emb_reg, clusters_emb_reg)
        gamma_reg     = torch.sum(inner_pro_reg, dim=1)
        return gamma, gamma_reg
    
    def getUsersPartRating_t(self, users, items, all_users, all_items):

        users_emb = all_users[users.long()]       
        items_emb = all_items[items.long()]
        
        rating = self.f(torch.mul(users_emb, items_emb).sum(dim=1))
        
        return rating   

In [13]:
def print_tense(x):
    return round(float(x),3)

# Preprocessing + Training 

In [ ]:
is_GNN = 1 # lgc-based model: 1, mf-based model:0
for Expe_index in [2,3]:
    data_path = './Datasets/'

    Expe_scenarios = ["MLtoAM", "AMtoML", "MLtoAB", "ABtoML", "AMtoAB", "ABtoAM"]
    source_file = ["Rating_MLasS.csv", "Rating_AmzMasS.csv", "Rating_MLasS.csv", "Rating_AmzBasS.csv", "Rating_AmzMasS.csv","Rating_AmzBasS.csv"]
    target_file = ["Rating_AmzMasT.csv", "Rating_MLasT.csv", "Rating_AmzBasT.csv", "Rating_MLasT.csv","Rating_AmzBasT.csv", "Rating_AmzMas.csv"]


    print("Experiment scenario: "+ Expe_scenarios[Expe_index])
    print("Source domain file: "+ source_file[Expe_index])
    print("Target domain file:" + target_file[Expe_index])

    df_S = pd.read_csv(data_path+ source_file[Expe_index])
    df_T = pd.read_csv(data_path+ target_file[Expe_index])

    if Expe_index>=4:
        df_S["movieId"] = df_S.deal_id
        df_S["userId"] = df_S.account_id

    source_name = Expe_scenarios[Expe_index][:2]
    target_name = Expe_scenarios[Expe_index][-2:]

    if Expe_index == 1 or Expe_index ==3:
        df_S = df_S[["account_id", "deal_id"]]
        df_S.columns =["userId", "movieId"]

        df_T = df_T[["userId", "movieId"]]
        df_T.columns =["account_id", "deal_id"]

    dict_path = 'Dictionary/'

    dict_item_id2index_t = dict(zip(df_T.deal_id.unique(), np.arange(len(df_T.deal_id.unique()))))
    dict_user_id2index_t = dict(zip(df_T.account_id.unique(), np.arange(len(df_T.account_id.unique()))))

    dict_item_id2index_s = dict(zip(df_S.movieId.unique(), np.arange(len(df_S.movieId.unique()))))
    dict_user_id2index_s = dict(zip(df_S.userId.unique(), np.arange(len(df_S.userId.unique()))))

    dict_item2vec_s = np.load(dict_path + f'{source_name}to{target_name}/Dict_item2vec_{source_name}asS.npy', allow_pickle=True).item()
    vec_matrix = [dict_item2vec_s[deal_id] for deal_id in df_S.movieId.unique()]
    dict_ItemIndex2vec_s = dict(zip(np.arange(len(df_S.movieId.unique())), vec_matrix))

    dict_item2vec_t = np.load(dict_path + f'{source_name}to{target_name}/Dict_item2vec_{target_name}asT.npy', allow_pickle=True).item()
    vec_matrix = [dict_item2vec_t[deal_id] for deal_id in df_T.deal_id.unique()]
    dict_ItemIndex2vec_t = dict(zip(np.arange(len(df_T.deal_id.unique())), vec_matrix))

    df_T["account_index"] = df_T.account_id.map(lambda x: dict_user_id2index_t[x])
    df_T["deal_index"] = df_T.deal_id.map(lambda x: dict_item_id2index_t[x])

    df_S["account_index"] = df_S.userId.map(lambda x: dict_user_id2index_s[x])
    df_S["deal_index"] = df_S.movieId.map(lambda x: dict_item_id2index_s[x])

    def ToList(x):
        return list(x)
    dict_interactions = dict(df_T.groupby(df_T["account_index"])["deal_index"].apply(ToList))
    dict_interactions_s = dict(df_S.groupby(df_S["account_index"])["deal_index"].apply(ToList))

    n_users_t = len(df_T.account_index.unique())
    m_items_t = len(df_T.deal_index.unique())
    n_inters_t = df_T.shape[0]

    n_users_s = len(df_S.account_index.unique())
    m_items_s = len(df_S.deal_index.unique())
    n_inters_s = df_S.shape[0]

    print(f"[Source domain]  n_users:{n_users_s}, m_items:{m_items_s}, n=inter.:{n_inters_s}")
    print(f"[Target domain]  n_users:{n_users_t}, m_items:{m_items_t}, n=inter.:{n_inters_t}")

    # Build graph

    tr_u_s, val_u_s, t_u_s = [], [], []
    tr_v_s, val_v_s, t_v_s = [], [], []
    for users in dict_interactions_s.keys():
        tr_u_s.extend([users] * len(dict_interactions_s[users]))
        tr_v_s.extend(dict_interactions_s[users])


    tr_u_s= np.array(tr_u_s)
    tr_v_s= np.array(tr_v_s)

    UseritemNet_s = csr_matrix((np.ones(len(tr_u_s)), (tr_u_s, tr_v_s)), shape=(n_users_s, m_items_s))

    tr_u_t, val_u_t, t_u_t = [], [], []
    tr_v_t, val_v_t, t_v_t = [], [], []
    for users in dict_interactions.keys():
        tr_u_t.extend([users] * len(dict_interactions[users][:-2]))
        val_u_t.append(users)
        t_u_t.append(users)

        tr_v_t.extend(dict_interactions[users][:-2])
        val_v_t.append(dict_interactions[users][-2])
        t_v_t.append(dict_interactions[users][-1])

    tr_u_t, val_u_t, t_u_t = np.array(tr_u_t), np.array(val_u_t), np.array(t_u_t)
    tr_v_t, val_v_t, t_v_t = np.array(tr_v_t), np.array(val_v_t), np.array(t_v_t)

    UseritemNet_t = csr_matrix((np.ones(len(tr_u_t)), (tr_u_t, tr_v_t)), shape=(n_users_t, m_items_t))



    config={}
    config['bpr_batch_size'] = 2048
    config['latent_dim_rec'] = 32
    config['lightGCN_n_layers_t']= 3
    config['lightGCN_n_layers_s']= 3
    config['test_u_batch_size'] = 100
    config['lr'] = 0.01
    config['decay_reg'] = 0.01 # beta Regularization loss
    config['decay_dp'] = 1 # gamma Reconstruction loss
    config['decay_kt'] = 0.1 #alpha Restriction loss
    config["seed"] = 6298
    config["epochs"] = 2000
    config["n_cluster"] = 32
    config["tao"] = 0.0001
    config["gamma_kt"] = 0.1
    #----------------------------------
    config['dropout'] = 0
    config['keep_prob']  = 0.6
    config['A_n_fold'] = 100
    config['multicore'] = 0
    config["topks"] = [20]

    UIGraph_s = getSparseGraph(n_users_s, m_items_s, UseritemNet_s)
    UIGraph_t = getSparseGraph(n_users_t, m_items_t, UseritemNet_t)
    if is_GNN:
        weight_file = f"{Expe_scenarios[Expe_index]}-SRTrans-wo_dp-LGC-Reg_{config['decay_reg']}-Rec_{config['decay_dp']}-Kt_{config['decay_kt']}-{config['latent_dim_rec']}.pth.tar"
    else:
        weight_file = f"{Expe_scenarios[Expe_index]}-SRTrans-wo_dp-MF-Reg_{config['decay_reg']}-Rec_{config['decay_dp']}-Kt_{config['decay_kt']}-{config['latent_dim_rec']}.pth.tar"
    Recmodel = SRTrans(config,UIGraph_s, UIGraph_t, is_GNN)
    #Recmodel = SRTrans(config)
    patient = 8
    min_ = 0.

    for epoch in range(config["epochs"]):

        if epoch %10 == 0 and epoch!=0:
            print("")
            results = Test(1)
            print("[TEST] recall:{0}, ndcg:{1}".format(results['recall'][0], results['ndcg'][0]))

            if results['ndcg'][0] > min_:
                torch.save(Recmodel.state_dict(), weight_file)
                min_ = results['ndcg'][0]
                patient = 8
                continue

            if results['ndcg'][0] <= min_:
                patient = patient - 1

            if patient == 0:                      
                break

        start = time.time()
        aver_loss, aver_pre_loss_s, aver_pre_loss_t, aver_rec_loss_s, aver_rec_loss_t, aver_kt_loss, aver_reg_loss = BPR_train_original(epoch)
        end = time.time()
        sys.stdout.write("\r ||epoch:{0}||loss:{1}||pre_loss_s:{2}||pre_loss_t:{3}||rec_loss_s:{4}||rec_loss_t:{5}||kt_loss:{6}||reg_loss:{7}||time:{8}".format(
            epoch, print_tense(aver_loss), print_tense(aver_pre_loss_s), print_tense(aver_pre_loss_t), 
            print_tense(aver_rec_loss_s), print_tense(aver_rec_loss_t), print_tense(aver_kt_loss), print_tense(aver_reg_loss), print_tense(end-start)))
        sys.stdout.flush()



# Evaluation

In [8]:
def Evaluation(Reclist, groundtruth, t_u, K):
    hit_rite = []
    ndcg = []
    for i in K:
        hr_i = 0.
        ndcg_i = 0.
        for j in range(len(t_u)):
            hr_i += len(set(Reclist[t_u[j]][:i+1]) & set([groundtruth[j]]))
            ndcg_i += getNDCG(Reclist[t_u[j]][:i+1], [groundtruth[j]])
        hit_rite.append(hr_i / len(groundtruth))
        ndcg.append(ndcg_i / len(groundtruth))
    return hit_rite, ndcg
def getDCG(scores):
    return np.sum(
        np.divide(np.power(2, scores) - 1, np.log(np.arange(scores.shape[0], dtype=np.float32) + 2)),
        dtype=np.float32)
def getNDCG(rank_list, pos_items):
    relevance = np.ones_like(pos_items)
    it2rel = {it: r for it, r in zip(pos_items, relevance)}
    rank_scores = np.asarray([it2rel.get(it, 0.0) for it in rank_list], dtype=np.float32)

    idcg = getDCG(relevance)

    dcg = getDCG(rank_scores)

    if dcg == 0.0:
        return 0.0

    ndcg = dcg / idcg
    return ndcg

In [9]:
import scipy.stats
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

def output_result(data):
    data = np.array(data).T
    for i in range(len(data)):
        m, h = mean_confidence_interval(data[i], confidence=0.95)
        m = round(m,3)
        h = round(h,3)
        print(f"{m} ± {h}  ", end=" ")
        #print(m, end=", ")

In [ ]:
for Expe_index in [2,3]:
    work_path = '/home/lizhi/Experiment_4_CIKM/'
    data_path = work_path + 'Datasets/'

    Expe_scenarios = ["MLtoAM", "AMtoML", "MLtoAB", "ABtoML", "AMtoAB", "ABtoAM"]
    source_file = ["Rating_MLasS.csv", "Rating_AmzMasS.csv", "Rating_MLasS.csv", "Rating_AmzBasS.csv", "Rating_AmzMasS.csv","Rating_AmzBasS.csv"]
    target_file = ["Rating_AmzMasT.csv", "Rating_MLasT.csv", "Rating_AmzBasT.csv", "Rating_MLasT.csv","Rating_AmzBasT.csv", "Rating_AmzMas.csv"]


    print("Experiment scenario: "+ Expe_scenarios[Expe_index])
    print("Source domain file: "+ source_file[Expe_index])
    print("Target domain file:" + target_file[Expe_index])

    df_S = pd.read_csv(data_path+ source_file[Expe_index])
    df_T = pd.read_csv(data_path+ target_file[Expe_index])

    if Expe_index>=4:
        df_S["movieId"] = df_S.deal_id
        df_S["userId"] = df_S.account_id

    source_name = Expe_scenarios[Expe_index][:2]
    target_name = Expe_scenarios[Expe_index][-2:]

    if Expe_index == 1 or Expe_index ==3:
        df_S = df_S[["account_id", "deal_id"]]
        df_S.columns =["userId", "movieId"]

        df_T = df_T[["userId", "movieId"]]
        df_T.columns =["account_id", "deal_id"]

    dict_path = work_path + 'Dictionary/'

    dict_item_id2index_t = dict(zip(df_T.deal_id.unique(), np.arange(len(df_T.deal_id.unique()))))
    dict_user_id2index_t = dict(zip(df_T.account_id.unique(), np.arange(len(df_T.account_id.unique()))))

    dict_item_id2index_s = dict(zip(df_S.movieId.unique(), np.arange(len(df_S.movieId.unique()))))
    dict_user_id2index_s = dict(zip(df_S.userId.unique(), np.arange(len(df_S.userId.unique()))))

    dict_item2vec_s = np.load(dict_path + f'{source_name}to{target_name}/Dict_item2vec_{source_name}asS.npy', allow_pickle=True).item()
    vec_matrix = [dict_item2vec_s[deal_id] for deal_id in df_S.movieId.unique()]
    dict_ItemIndex2vec_s = dict(zip(np.arange(len(df_S.movieId.unique())), vec_matrix))

    dict_item2vec_t = np.load(dict_path + f'{source_name}to{target_name}/Dict_item2vec_{target_name}asT.npy', allow_pickle=True).item()
    vec_matrix = [dict_item2vec_t[deal_id] for deal_id in df_T.deal_id.unique()]
    dict_ItemIndex2vec_t = dict(zip(np.arange(len(df_T.deal_id.unique())), vec_matrix))

    df_T["account_index"] = df_T.account_id.map(lambda x: dict_user_id2index_t[x])
    df_T["deal_index"] = df_T.deal_id.map(lambda x: dict_item_id2index_t[x])

    df_S["account_index"] = df_S.userId.map(lambda x: dict_user_id2index_s[x])
    df_S["deal_index"] = df_S.movieId.map(lambda x: dict_item_id2index_s[x])

    def ToList(x):
        return list(x)
    dict_interactions = dict(df_T.groupby(df_T["account_index"])["deal_index"].apply(ToList))
    dict_interactions_s = dict(df_S.groupby(df_S["account_index"])["deal_index"].apply(ToList))

    n_users_t = len(df_T.account_index.unique())
    m_items_t = len(df_T.deal_index.unique())
    n_inters_t = df_T.shape[0]

    n_users_s = len(df_S.account_index.unique())
    m_items_s = len(df_S.deal_index.unique())
    n_inters_s = df_S.shape[0]

    print(f"[Source domain]  n_users:{n_users_s}, m_items:{m_items_s}, n=inter.:{n_inters_s}")
    print(f"[Target domain]  n_users:{n_users_t}, m_items:{m_items_t}, n=inter.:{n_inters_t}")

    # Build graph

    tr_u_s, val_u_s, t_u_s = [], [], []
    tr_v_s, val_v_s, t_v_s = [], [], []
    for users in dict_interactions_s.keys():
        tr_u_s.extend([users] * len(dict_interactions_s[users]))
        tr_v_s.extend(dict_interactions_s[users])


    tr_u_s= np.array(tr_u_s)
    tr_v_s= np.array(tr_v_s)

    UseritemNet_s = csr_matrix((np.ones(len(tr_u_s)), (tr_u_s, tr_v_s)), shape=(n_users_s, m_items_s))

    tr_u_t, val_u_t, t_u_t = [], [], []
    tr_v_t, val_v_t, t_v_t = [], [], []
    for users in dict_interactions.keys():
        tr_u_t.extend([users] * len(dict_interactions[users][:-2]))
        val_u_t.append(users)
        t_u_t.append(users)

        tr_v_t.extend(dict_interactions[users][:-2])
        val_v_t.append(dict_interactions[users][-2])
        t_v_t.append(dict_interactions[users][-1])

    tr_u_t, val_u_t, t_u_t = np.array(tr_u_t), np.array(val_u_t), np.array(t_u_t)
    tr_v_t, val_v_t, t_v_t = np.array(tr_v_t), np.array(val_v_t), np.array(t_v_t)

    UseritemNet_t = csr_matrix((np.ones(len(tr_u_t)), (tr_u_t, tr_v_t)), shape=(n_users_t, m_items_t))
    
    UIGraph_s = getSparseGraph(n_users_s, m_items_s, UseritemNet_s)
    UIGraph_t = getSparseGraph(n_users_t, m_items_t, UseritemNet_t)
    
    item_list = df_T.deal_index.unique()
    

    if is_GNN:
        weight_file = f"{Expe_scenarios[Expe_index]}-SRTrans-wo_dp-LGC-Reg_{config['decay_reg']}-Rec_{config['decay_dp']}-Kt_{config['decay_kt']}-{config['latent_dim_rec']}.pth.tar"
    else:
        weight_file = f"{Expe_scenarios[Expe_index]}-SRTrans-wo_dp-MF-Reg_{config['decay_reg']}-Rec_{config['decay_dp']}-Kt_{config['decay_kt']}-{config['latent_dim_rec']}.pth.tar"
    
    model = SRTrans(config, UIGraph_s, UIGraph_t, is_GNN)
    model.load_state_dict(torch.load(weight_file)) 
    model.eval()

    HR = []
    NDCG = []
    
    all_users, all_items = model.computer_t()
    for i in range(10):
        np.random.seed(i*7) 
        condidate_item = np.random.choice(item_list, 120, replace=False)

        pred_reclist = {}
        item_matrix = []
        for index in tqdm(range(len(t_u_t))):
            u = t_u_t[index]
            u_condidate_item = np.setdiff1d(condidate_item, dict_interactions[u])
            u_condidate_item = np.random.choice(u_condidate_item, 99, replace=False)
            u_condidate_item = np.union1d(u_condidate_item, dict_interactions[u][-1:])
            item_matrix.append(u_condidate_item)
        v = torch.LongTensor(item_matrix)
        v = v.t()
        pred_result = []

        for j in tqdm(range(100)):
            y = model.getUsersPartRating_t(torch.LongTensor(t_u_t).cuda(), v[j].cuda(), all_users, all_items).cpu()
            pred_result.append(list(y.data))

        result = torch.Tensor(pred_result).t()
        #result = pred_result.t()
        topk_indices = np.array(torch.topk(result, 100, dim=1).indices)

        topk_items = []
        item_matrix = np.array(item_matrix)
        for i in range(len(topk_indices)):
            topk_items.append(item_matrix[i][topk_indices[i]])
        topk_items = np.array(topk_items)

        pred_reclist = dict(zip(t_u_t, topk_items))

        hit_rite, ndcg = Evaluation(pred_reclist, t_v_t, t_u_t, [i for i in range(10)])
        HR.append(hit_rite)
        NDCG.append(ndcg)
    output_result(HR)
    print()
    output_result(NDCG)